In [43]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

In [138]:
df = pd.read_csv('heart_3datasets_comimp_aggressive_pca_best_StrategyA_AggCorr85.csv')
df.loc[df['target'] == 2, 'target'] = 1

X = df.drop('target', axis=1)
y = df['target']


df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PC1_AggCorr85  2919 non-null   float64
 1   PC2_AggCorr85  2919 non-null   float64
 2   trestbps       2919 non-null   float64
 3   chol           2919 non-null   float64
 4   sex            2919 non-null   int64  
 5   cp             2919 non-null   int64  
 6   fbs            2919 non-null   int64  
 7   restecg        2919 non-null   int64  
 8   exang          2919 non-null   int64  
 9   slope          2919 non-null   int64  
 10  ca             2919 non-null   int64  
 11  thal           2919 non-null   int64  
 12  target         2919 non-null   int64  
dtypes: float64(4), int64(9)
memory usage: 296.6 KB


In [139]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_trainval.head()

,PC1_AggCorr85,PC2_AggCorr85,trestbps,chol,sex,cp,fbs,restecg,exang,slope,ca,thal
2159,-0.256919,1.769050,104.0,208.0,1,1,0,0,1,2,0,2
176,-0.426075,0.109233,117.0,230.0,1,1,1,1,1,3,2,3
2624,1.839476,-0.054307,120.0,220.0,1,2,0,1,0,3,0,2
2329,0.261295,-0.228538,120.0,330.0,0,1,0,1,1,3,0,2
1215,-2.236837,0.452950,158.0,210.0,1,4,1,0,1,3,0,2


In [140]:
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval)
y_train.value_counts()

X_trainval.head()

,PC1_AggCorr85,PC2_AggCorr85,trestbps,chol,sex,cp,fbs,restecg,exang,slope,ca,thal
2159,-0.256919,1.769050,104.0,208.0,1,1,0,0,1,2,0,2
176,-0.426075,0.109233,117.0,230.0,1,1,1,1,1,3,2,3
2624,1.839476,-0.054307,120.0,220.0,1,2,0,1,0,3,0,2
2329,0.261295,-0.228538,120.0,330.0,0,1,0,1,1,3,0,2
1215,-2.236837,0.452950,158.0,210.0,1,4,1,0,1,3,0,2


In [142]:
#phase 3 comment cat_features
#use this for phase 4
cat_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

#use this for phase 1,2
# cat_features = ['sex', 'Chest pain type', 'fasting blood pressure', 'resting ecg', 'exercise angina', 'ST slope']



categorical_indices = []
for feature in cat_feature:
    if feature in X.columns:
        categorical_indices.append(X.columns.get_loc(feature))

from catboost import Pool, cv, CatBoostClassifier

print(f"Categorical feature indices: {categorical_indices}")
print(f"Categorical features for CatBoost: {[X.columns[i] for i in categorical_indices]}")
# Create and train CatBoost model
print("Training CatBoost model...")

# Initialize CatBoost classifier
# catboost_model = CatBoostClassifier(
#     cat_features=categorical_indices,
#     eval_metric='AUC',
#     random_seed=42
# )


train_pool = Pool(data=X_trainval,
                  label=y_trainval, cat_features=cat_features)

train_pool = Pool(data=X_trainval,
                  label=y_trainval, cat_features=cat_features)

params = {
    'eval_metric': 'AUC',
    'random_seed': 42,
    'loss_function': 'Logloss',
}

cv_results = cv(
    pool=train_pool,
    params=params,
    fold_count=5,  # Number of folds
    stratified=True, # Use stratified sampling for classification
    as_pandas=True   # Return results as a pandas DataFrame
)

# 5. Display the results
print("CV results:")
print(cv_results.tail()) # Show the last few iterations

print("\nBest validation AUC score:")
print(cv_results['test-AUC-mean'].max())



Categorical feature indices: [4]
Categorical features for CatBoost: ['sex']
Training CatBoost model...
Training on fold [0/5]
0:	test: 0.7768545	best: 0.7768545 (0)	total: 44.9ms	remaining: 44.9s
1:	test: 0.8005148	best: 0.8005148 (1)	total: 105ms	remaining: 52.5s
2:	test: 0.8103134	best: 0.8103134 (2)	total: 150ms	remaining: 49.9s
3:	test: 0.8108558	best: 0.8108558 (3)	total: 197ms	remaining: 49.1s
4:	test: 0.8133100	best: 0.8133100 (4)	total: 244ms	remaining: 48.6s
5:	test: 0.8305083	best: 0.8305083 (5)	total: 291ms	remaining: 48.1s
6:	test: 0.8467598	best: 0.8467598 (6)	total: 337ms	remaining: 47.9s
7:	test: 0.8501517	best: 0.8501517 (7)	total: 384ms	remaining: 47.6s
8:	test: 0.8492968	best: 0.8501517 (7)	total: 436ms	remaining: 48s
9:	test: 0.8516132	best: 0.8516132 (9)	total: 543ms	remaining: 53.8s
10:	test: 0.8564942	best: 0.8564942 (10)	total: 614ms	remaining: 55.2s
11:	test: 0.8581212	best: 0.8581212 (11)	total: 674ms	remaining: 55.5s
12:	test: 0.8598401	best: 0.8598401 (12)	to

In [73]:
cv_score.mean()

0.856875778977981

In [143]:
# # Train the model
catboost_model = CatBoostClassifier(**params)

from imblearn.over_sampling import SMOTENC
smote_enc = SMOTENC(random_state=42, categorical_features=cat_features)
X_train, y_train = smote_enc.fit_resample(X_train, y_train)

catboost_model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    plot=True,
    use_best_model=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.03769
0:	test: 0.7849668	best: 0.7849668 (0)	total: 1.59ms	remaining: 1.59s
1:	test: 0.8253507	best: 0.8253507 (1)	total: 2.96ms	remaining: 1.48s
2:	test: 0.8359819	best: 0.8359819 (2)	total: 4.37ms	remaining: 1.45s
3:	test: 0.8408730	best: 0.8408730 (3)	total: 5.88ms	remaining: 1.47s
4:	test: 0.8446382	best: 0.8446382 (4)	total: 7.63ms	remaining: 1.52s
5:	test: 0.8500092	best: 0.8500092 (5)	total: 9.26ms	remaining: 1.53s
6:	test: 0.8446290	best: 0.8500092 (5)	total: 10.8ms	remaining: 1.54s
7:	test: 0.8490772	best: 0.8500092 (5)	total: 12.6ms	remaining: 1.56s
8:	test: 0.8522148	best: 0.8522148 (8)	total: 14.2ms	remaining: 1.57s
9:	test: 0.8511074	best: 0.8522148 (8)	total: 15.7ms	remaining: 1.55s
10:	test: 0.8578165	best: 0.8578165 (10)	total: 17.2ms	remaining: 1.55s
11:	test: 0.8602252	best: 0.8602252 (11)	total: 18.9ms	remaining: 1.55s
12:	test: 0.8592839	best: 0.8602252 (11)	total: 20.7ms	remaining: 1.57s
13:	test: 0.8578627	best: 0.8602252 (11)	total: 22.1ms	

In [145]:
y_pred = catboost_model.predict(X_test)
accuracy_score(y_pred, y_test)
roc_auc_score(y_pred, y_test)

0.8713984081230798

In [91]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [132]:
print('F1 score: ', f1_score(y_pred, y_test))
print('Accuracy score: ', accuracy_score(y_pred, y_test))
print('Precision score: ', precision_score(y_pred, y_test))
print('Recall score: ', recall_score(y_pred, y_test))
print('roc-auc', roc_auc_score(y_pred, y_test))

F1 score:  0.8691588785046729
Accuracy score:  0.8561643835616438
Precision score:  0.8829113924050633
Recall score:  0.8558282208588958
roc-auc 0.8562086840728587


In [146]:
print('F1 score: ', f1_score(y_pred, y_test))
print('Accuracy score: ', accuracy_score(y_pred, y_test))
print('Precision score: ', precision_score(y_pred, y_test))
print('Recall score: ', recall_score(y_pred, y_test))
print('roc-auc', roc_auc_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

F1 score:  0.8826291079812206
Accuracy score:  0.8715753424657534
Precision score:  0.8924050632911392
Recall score:  0.8730650154798761
roc-auc 0.8713984081230798
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       261
           1       0.89      0.87      0.88       323

    accuracy                           0.87       584
   macro avg       0.87      0.87      0.87       584
weighted avg       0.87      0.87      0.87       584



In [147]:
# 2. Define the objective function with pruning

from optuna.integration import CatBoostPruningCallback # Import the callback
def objective_pruning(trial):
    # Define the hyperparameter search space (same as before)
    params = {
        'iterations': 1000, # Use a high number, pruning will handle it
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'cat_features': cat_features,
        'logging_level': 'Silent',
        'random_seed': 42,
        'early_stopping_rounds': 50 # Required for pruning
    }
    
    # Initialize the model
    model = CatBoostClassifier(**params)
    
    # Create the pruning callback
    # This will report the 'AUC' on the validation set to Optuna
    pruning_callback = CatBoostPruningCallback(trial, 'AUC')
    
    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[pruning_callback] # Pass the callback here
    )
    
    # Manually check if the trial was pruned
    pruning_callback.check_pruned()
    
    # Get the score from the best iteration
    score = model.get_best_score()['validation']['AUC']
    return score

# 3. Create and run the study
study_pruning = optuna.create_study(direction='maximize')
study_pruning.optimize(objective_pruning, n_trials=200) # Can run more trials in the same amount of time

# 4. Print the best results
print("\n--- Best results from Pruning-based tuning ---")
print(f"Best trial score: {study_pruning.best_value:.4f}")
print("Best hyperparameters:")
print(study_pruning.best_params)

[I 2025-09-21 20:41:38,724] A new study created in memory with name: no-name-4b4a358d-2893-4483-86d5-0946df77df32
C:\Users\PC\AppData\Local\Temp\ipykernel_34904\2829041095.py:26: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, 'AUC')
[I 2025-09-21 20:41:58,386] Trial 0 finished with value: 0.9170542635658915 and parameters: {'depth': 10, 'learning_rate': 0.15455123294207693, 'l2_leaf_reg': 1.6136238624901977, 'bagging_temperature': 0.23315568604788017, 'random_strength': 7.712307174358268}. Best is trial 0 with value: 0.9170542635658915.
[I 2025-09-21 20:42:05,966] Trial 1 finished with value: 0.9183093392395718 and parameters: {'depth': 8, 'learning_rate': 0.06180903509697781, 'l2_leaf_reg': 0.3663626655083058, 'bagging_temperature': 0.594823573327259, 'random_strength': 0.004143101703707347}. Best is trial 1 with value: 0.9183093392395718.
[I 2025-09-21 20


--- Best results from Pruning-based tuning ---
Best trial score: 0.9279
Best hyperparameters:
{'depth': 9, 'learning_rate': 0.09315483760483033, 'l2_leaf_reg': 2.364479598460067, 'bagging_temperature': 0.2851951290066055, 'random_strength': 0.004067038247310275}


In [148]:
final_model = CatBoostClassifier(**study_pruning.best_params)
final_model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True, use_best_model=True)
print("\nFinal model trained with best parameters.")
predict = final_model.predict(X_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6246081	test: 0.6322387	best: 0.6322387 (0)	total: 4.45ms	remaining: 4.44s
1:	learn: 0.5708918	test: 0.5891164	best: 0.5891164 (1)	total: 9.15ms	remaining: 4.57s
2:	learn: 0.5248329	test: 0.5560199	best: 0.5560199 (2)	total: 13.9ms	remaining: 4.63s
3:	learn: 0.4861871	test: 0.5232524	best: 0.5232524 (3)	total: 17.9ms	remaining: 4.45s
4:	learn: 0.4546987	test: 0.5041526	best: 0.5041526 (4)	total: 21.9ms	remaining: 4.35s
5:	learn: 0.4291767	test: 0.4810627	best: 0.4810627 (5)	total: 27ms	remaining: 4.47s
6:	learn: 0.4082230	test: 0.4664321	best: 0.4664321 (6)	total: 31.4ms	remaining: 4.45s
7:	learn: 0.3866196	test: 0.4559565	best: 0.4559565 (7)	total: 35.5ms	remaining: 4.4s
8:	learn: 0.3708881	test: 0.4443028	best: 0.4443028 (8)	total: 39.8ms	remaining: 4.38s
9:	learn: 0.3532892	test: 0.4360389	best: 0.4360389 (9)	total: 43.9ms	remaining: 4.35s
10:	learn: 0.3378438	test: 0.4298618	best: 0.4298618 (10)	total: 48.1ms	remaining: 4.33s
11:	learn: 0.3272758	test: 0.4220751	best: 0

In [149]:
accuracy_score(predict, y_test)

0.8715753424657534

In [150]:
print('F1 score: ', f1_score(predict, y_test))
print('Accuracy score: ', accuracy_score(predict, y_test))
print('Precision score: ', precision_score(predict, y_test))
print('Recall score: ', recall_score(predict, y_test))
print('roc-auc', roc_auc_score(predict, y_test))
print(classification_report(predict, y_test))

F1 score:  0.8807631160572337
Accuracy score:  0.8715753424657534
Precision score:  0.8765822784810127
Recall score:  0.8849840255591054
roc-auc 0.870536293222357
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       271
           1       0.88      0.88      0.88       313

    accuracy                           0.87       584
   macro avg       0.87      0.87      0.87       584
weighted avg       0.87      0.87      0.87       584



In [151]:
study_pruning.best_params

{'depth': 9,
 'learning_rate': 0.09315483760483033,
 'l2_leaf_reg': 2.364479598460067,
 'bagging_temperature': 0.2851951290066055,
 'random_strength': 0.004067038247310275}